In [63]:
import pandas as pd
from textacy import preprocessing

df2 = pd.read_csv('Batch_5298950_batch_results.csv')


df2 = df2[['Input.text', 'Answer.category.labels']]
# for all labels in Answer.category.labels containing a comma, split them into separate rows
df2['Answer.category.labels'] = df2['Answer.category.labels'].str.split(',')
df2 = df2.explode('Answer.category.labels')

df = df2

# remove Answer.category.labels with label count less than 2
df = df.groupby('Answer.category.labels').filter(lambda x: len(x) > 1)
print(df['Answer.category.labels'].value_counts())
# turn into one-hot encoding
# remove not about vaccines label
df = df[df['Answer.category.labels'] != 'Not about vaccines']
# print unique labels
print(df['Answer.category.labels'].unique())
df['Answer.category.labels'] = df['Answer.category.labels'].astype('category').cat.codes
# merge rows on Input.text
#df = df.groupby('Input.text').sum()

# divide by sum of dummy columns to get percentage
#df = df.div(df.sum(axis=1), axis=0)

df3 = pd.read_csv("Batch_5298954_batch_results.csv")
df3 = df3[['Input.text', 'Answer.category.labels']]
# remove \ from Answer.category.labels
df3['Answer.category.labels'] = df3['Answer.category.labels'].str.replace('\\', '')
df3['Answer.category.labels'] = df3['Answer.category.labels'].str.split(',')
# combine "Vaccination schedule", "spacing", "or timing" into one category
df3['Answer.category.labels'] = df3['Answer.category.labels'].apply(lambda x: ['Vaccination schedule, spacing, or timing' if y in ['Vaccination schedule', ' spacing', ' or timing'] else y for y in x])
df3 = df3.explode('Answer.category.labels')
df3 = df3.groupby('Answer.category.labels').filter(lambda x: len(x) > 1)
print(df3['Answer.category.labels'].value_counts())
df3 = df3[df3['Answer.category.labels'] != 'Not about vaccines']
print(df3['Answer.category.labels'].unique())
df3['Answer.category.labels'] = df3['Answer.category.labels'].astype('category').cat.codes
#df3 = df3.groupby('Input.text').sum()
#df3 = df3.div(df3.sum(axis=1), axis=0)


Answer.category.labels
Vaccinated/planning to vaccinate                        529
No information about the author’s vaccine intentions    394
Not vaccinated/not planning to vaccinate                 97
Not about vaccines                                       26
Name: count, dtype: int64
['Vaccinated/planning to vaccinate'
 'No information about the author’s vaccine intentions'
 'Not vaccinated/not planning to vaccinate']
Answer.category.labels
Vaccination schedule, spacing, or timing                          684
Vaccines are mentioned positively                                 310
Protection from disease                                           292
About vaccines but does not fit any of the previous categories    278
Adverse effects                                                   277
Vaccines are mentioned negatively                                 126
Seeking information about vaccines                                110
Not about vaccines                                           

In [59]:
# map preprocessing.remove.punctuation to all Input.text
df['Input.text'] = df['Input.text'].map(preprocessing.remove.punctuation)
df3['Input.text'] = df3['Input.text'].map(preprocessing.remove.punctuation)

df['Input.text'] = df['Input.text'].map(preprocessing.normalize.whitespace)
df3['Input.text'] = df3['Input.text'].map(preprocessing.normalize.whitespace)

df['Input.text'] = df['Input.text'].map(preprocessing.replace.urls)
df3['Input.text'] = df3['Input.text'].map(preprocessing.replace.urls)

df['Input.text'] = df['Input.text'].map(preprocessing.replace.emails)
df3['Input.text'] = df3['Input.text'].map(preprocessing.replace.emails)

df['Input.text'] = df['Input.text'].map(preprocessing.replace.phone_numbers)
df3['Input.text'] = df3['Input.text'].map(preprocessing.replace.phone_numbers)



In [60]:
# rename Input.text to text
df.rename(columns={'Input.text': 'text'}, inplace=True)
df3.rename(columns={'Input.text': 'text'}, inplace=True)

# rename Answer.category.labels to label
df.rename(columns={'Answer.category.labels': 'label'}, inplace=True)
df3.rename(columns={'Answer.category.labels': 'label'}, inplace=True)

In [61]:
# drop duplicates
df = df.drop_duplicates()
df3 = df3.drop_duplicates()



In [62]:
df.to_csv('stance_labels.csv')
df3.to_csv('behavior_labels.csv')